In [1]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

rng = np.random.default_rng(42)

# -----------------------------
# 1. Define stimulus space
# -----------------------------
# 2D inputs: (s1, s2)
N_STIM = 2000
s1 = rng.uniform(-1, 1, size=N_STIM)
s2 = rng.uniform(-1, 1, size=N_STIM)
S = np.stack([s1, s2], axis=1)   # shape (N_STIM, 2)

# XOR-ish labels: 1 if s1 and s2 have same sign, else 0
y = ((s1 * s2) > 0).astype(int)

# -----------------------------
# 2. Build three populations
# -----------------------------
N_NEURONS = 200

def relu(x):
    return np.maximum(0, x)

# (a) Pure selectivity: mostly s1 or mostly s2
def make_pure_population(S, n_neurons):
    s1, s2 = S[:, 0], S[:, 1]
    pops = []
    for _ in range(n_neurons):
        if rng.random() < 0.5:
            # neuron of type "mostly s1"
            w1 = rng.normal(1.0, 0.1)
            w2 = rng.normal(0.0, 0.1)
        else:
            # neuron of type "mostly s2"
            w1 = rng.normal(0.0, 0.1)
            w2 = rng.normal(1.0, 0.1)
        b = rng.normal(0.0, 0.2)
        r = relu(w1 * s1 + w2 * s2 + b)
        pops.append(r)
    return np.stack(pops, axis=1)  # (N_STIM, N_NEURONS)

# (b) Linear mixed selectivity
def make_linear_mixed_population(S, n_neurons):
    s1, s2 = S[:, 0], S[:, 1]
    pops = []
    for _ in range(n_neurons):
        w1 = rng.normal(0.0, 1.0)
        w2 = rng.normal(0.0, 1.0)
        b = rng.normal(0.0, 0.2)
        r = relu(w1 * s1 + w2 * s2 + b)
        pops.append(r)
    return np.stack(pops, axis=1)

# (c) Nonlinear mixed selectivity: includes interaction term s1*s2
def make_nonlinear_mixed_population(S, n_neurons):
    s1, s2 = S[:, 0], S[:, 1]
    pops = []
    for _ in range(n_neurons):
        w1 = rng.normal(0.0, 1.0)
        w2 = rng.normal(0.0, 1.0)
        w12 = rng.normal(1.0, 0.5)   # interaction weight
        b = rng.normal(0.0, 0.2)
        r = relu(w1 * s1 + w2 * s2 + w12 * (s1 * s2) + b)
        pops.append(r)
    return np.stack(pops, axis=1)

X_pure = make_pure_population(S, N_NEURONS)
X_lin  = make_linear_mixed_population(S, N_NEURONS)
X_non  = make_nonlinear_mixed_population(S, N_NEURONS)

# -----------------------------
# 3. PCA: effective dimensionality
# -----------------------------
def summarize_pca(X, name, var_threshold=0.9):
    pca = PCA()
    pca.fit(X)
    cum = np.cumsum(pca.explained_variance_ratio_)
    dim_90 = np.searchsorted(cum, var_threshold) + 1
    print(f"[PCA] {name}: dims for {var_threshold*100:.0f}% var = {dim_90}")
    return dim_90

dim_pure = summarize_pca(X_pure, "PURE")
dim_lin  = summarize_pca(X_lin,  "LINEAR MIXED")
dim_non  = summarize_pca(X_non,  "NONLINEAR MIXED")

# -----------------------------
# 4. Linear readout for XOR
# -----------------------------
def linear_readout_accuracy(X, y, name):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    clf = LogisticRegression(max_iter=1000)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"[XOR] {name}: linear readout accuracy = {acc:.3f}")
    return acc

acc_pure = linear_readout_accuracy(X_pure, y, "PURE")
acc_lin  = linear_readout_accuracy(X_lin,  y, "LINEAR MIXED")
acc_non  = linear_readout_accuracy(X_non,  y, "NONLINEAR MIXED")


[PCA] PURE: dims for 90% var = 2
[PCA] LINEAR MIXED: dims for 90% var = 4
[PCA] NONLINEAR MIXED: dims for 90% var = 3
[XOR] PURE: linear readout accuracy = 0.975
[XOR] LINEAR MIXED: linear readout accuracy = 0.993
[XOR] NONLINEAR MIXED: linear readout accuracy = 0.988


In [1]:
import numpy as np

# Two input features
x1 = np.random.randn(500)
x2 = np.random.randn(500)

# Pure selectivity neurons
n1 = x1
n2 = x2

# Mixed selectivity neuron (nonlinear!)
n3 = x1 * x2

print(np.corrcoef([n1, n2, n3]))


[[ 1.         -0.09653616 -0.01512067]
 [-0.09653616  1.          0.04602408]
 [-0.01512067  0.04602408  1.        ]]


In [2]:
import numpy as np

# inputs
x1 = np.random.randn(1000)
x2 = np.random.randn(1000)

# hidden units mix x1 and x2
h1 = np.tanh(1.2*x1 + 0.7*x2)
h2 = np.tanh(-0.3*x1 + 2.1*x2)

# output layer = linear readout of nonlinear mixtures
y = 0.5*h1 - 0.2*h2

print(np.corrcoef([x1, x2, y]))


[[1.         0.01035689 0.91379284]
 [0.01035689 1.         0.05036653]
 [0.91379284 0.05036653 1.        ]]
